## Notebook for testing Redbox app

In [ ]:
%load_ext dotenv
%dotenv ../redbox/.env.local

In [ ]:
from redbox.models.settings import Settings, get_settings
from redbox.models.chain import ChatLLMBackend, ChainChatMessage
from redbox.app import Redbox
from redbox.models.chain import RedboxQuery, RedboxState, AgentEnum, AgentTask, MultiAgentPlan
from uuid import uuid4
from redbox.models.chain import AISettings

def get_state(user_uuid, prompts, documents, ai_setting, permitted_s3_keys):
    q = RedboxQuery(
        question=f"{prompts[-1]}",
        s3_keys=documents,
        user_uuid=user_uuid,
        chat_history=[ChainChatMessage(role="user", text=s) for s in prompts[:-1]],
        ai_settings=ai_setting,
        permitted_s3_keys=permitted_s3_keys,
    )

    return RedboxState(
        request=q,
    )

def run_app(app, state) -> RedboxState:
    return app.graph.invoke(state)

def run_use_case(document, question, app):
    x = get_state(user_uuid=uuid4(), prompts=[question], documents=document, ai_setting=AISettings(chat_backend=ChatLLMBackend(name="anthropic.claude-3-7-sonnet-20250219-v1:0", provider="bedrock")), permitted_s3_keys=document)
    res = run_app(app, x)
    print('Final response:')
    print(res['messages'])
    return res

env = Settings()
app = Redbox(env=env, debug = True)

### Graph visualisation

In [ ]:
app.graph

### Running use cases


After planning

In [ ]:
plan = MultiAgentPlan(tasks=[AgentTask(id='task1', task="Analyze the user's documents to understand what recommendations are made in the AI Playbook for the UK Government", agent=AgentEnum.Internal_Retrieval_Agent, expected_output='A comprehensive list of key recommendations made in the AI Playbook document', dependencies=[]), AgentTask(id='task2', task="Search for public opinion surveys, reports, or articles regarding the UK Government's AI Playbook recommendations", agent=AgentEnum.Web_Search_Agent, expected_output="Recent data, surveys, or articles showing public sentiment towards the recommendations in the UK Government's AI Playbook", dependencies=['task1']), AgentTask(id='task3', task='Check if any government websites have published public consultation results or feedback related to the AI Playbook', agent=AgentEnum.External_Retrieval_Agent, expected_output='Information from gov.uk about any public consultations, feedback, or opinion polls related to the AI Playbook recommendations', dependencies=['task1'])])

question = """Does the public agree with the recommendation in this document?"""

document = ['test@dbt.gov.uk/AI_Playbook_for_the_UK_Government.pdf']
x = get_state(user_uuid=uuid4(), prompts=[question], documents=document, ai_setting=AISettings(chat_backend=ChatLLMBackend(name="anthropic.claude-3-7-sonnet-20250219-v1:0", provider="bedrock")), permitted_s3_keys=document)
x.user_feedback = "proceed"
x.agent_plans = plan
x.tasks_evaluator = ""

res = run_app(app, x)

#### Usecase 

In [ ]:
document = []
question = 'tell me about recent news on Trump visit to UK from bbc news'
res = run_use_case(document, question, app)


In [ ]:
document = ['test@dbt.gov.uk/AI_Playbook_for_the_UK_Government.pdf']
question = 'Sumarise this document and draft an email to my manager highlighting the key areas we need to focus when developing AI solution in the department'
res = run_use_case(document, question, app)